In [1]:

from google.colab import drive
drive.mount('/content/drive')
!pip3 install ipdb

Mounted at /content/drive
     |████████████████████████████████| 791 kB 6.3 MB/s 
     |████████████████████████████████| 374 kB 35.0 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=23d9f6fd9ab5f645d42248098746de38d96a805193219f617498e17864cb744d
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, bu

In [2]:
# !pwd
# %cd /content/

In [ ]:
!mkdir rob535data

!cp drive/Shareddrives/ROB\ 535\ -\ Perception/*data.zip /content/rob535data/

!unzip ./rob535data/*.zip

In [4]:
%cd drive/Shareddrives/ROB\ 535\ -\ Perception
!pwd

/content/drive/Shareddrives/ROB 535 - Perception
/content/drive/Shareddrives/ROB 535 - Perception


In [5]:
#!mkdir checkpoints
!pwd

/content/drive/Shareddrives/ROB 535 - Perception


In [ ]:
execfile("train.py")

=> Loading checkpoint
Loading Data...
Finish Loading.


100%|██████████| 236/236 [06:24<00:00,  1.63s/it, loss=1.02]


Mean loss was 1.2375313577763105


100%|██████████| 236/236 [06:22<00:00,  1.62s/it, loss=0.634]


Mean loss was 1.5011155400740899


...

100%|██████████| 236/236 [06:22<00:00,  1.62s/it, loss=1.51]


Mean loss was 3.6506415890940165


100%|██████████| 236/236 [06:22<00:00,  1.62s/it, loss=1.5]


Mean loss was 2.1463903817584957


 85%|████████▌ | 201/236 [05:26<00:56,  1.62s/it, loss=0.74]

In [ ]:
help(Yolov1.load_state_dict)

Help on function load_state_dict in module torch.nn.modules.module:

load_state_dict(self, state_dict: 'OrderedDict[str, Tensor]', strict: bool = True)
    Copies parameters and buffers from :attr:`state_dict` into
    this module and its descendants. If :attr:`strict` is ``True``, then
    the keys of :attr:`state_dict` must exactly match the keys returned
    by this module's :meth:`~torch.nn.Module.state_dict` function.
    
    Args:
        state_dict (dict): a dict containing parameters and
            persistent buffers.
        strict (bool, optional): whether to strictly enforce that the keys
            in :attr:`state_dict` match the keys returned by this module's
            :meth:`~torch.nn.Module.state_dict` function. Default: ``True``
    
    Returns:
        ``NamedTuple`` with ``missing_keys`` and ``unexpected_keys`` fields:
            * **missing_keys** is a list of str containing the missing keys
            * **unexpected_keys** is a list of str containing the une

In [ ]:
"""
Main file for training Yolo model on Pascal VOC dataset

"""

import torch
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from dataset import GTADataset
from utils.utils import (
    mean_average_precision,
    get_bboxes,
    load_checkpoint,
)
from loss import YoloLoss

seed = 123
torch.manual_seed(seed)

# Hyperparameters etc.
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 1000
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "data/images"
LABEL_DIR = "data/labels"


def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item())

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")


def main():
    model = Yolov1(split_size=7, num_boxes=2, num_classes=3).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss(S=7, B=2, C=3)

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = GTADataset('./data/trainval_labels.csv', './data/trainval_bboxes.csv')

    test_dataset = GTADataset('./data/trainval_labels.csv', 'data/trainval_bboxes.csv')

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    for epoch in range(EPOCHS):
        # for x, y in train_loader:
        #    x = x.to(DEVICE)
        #    for idx in range(8):
        #        bboxes = cellboxes_to_boxes(model(x))
        #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
        #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

        #    import sys
        #    sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
        print(f"Train mAP: {mean_avg_prec}")

        train_fn(train_loader, model, optimizer, loss_fn)

        #if mean_avg_prec > 0.9:
        #    checkpoint = {
        #        "state_dict": model.state_dict(),
        #        "optimizer": optimizer.state_dict(),
        #    }
        #    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        #    import time
        #    time.sleep(10)


if __name__ == "__main__":
    print("dsf")
    main()
    

dsf


KeyboardInterrupt: ignored

In [ ]:
model = Yolov1(split_size=7, num_boxes=2, num_classes=3).to(DEVICE)
optimizer = optim.Adam(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
)
loss_fn = YoloLoss(S=7, B=2, C=3)

if LOAD_MODEL:
    load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

train_dataset = GTADataset('./data/trainval_labels.csv', './data/trainval_bboxes.csv')

test_dataset = GTADataset('./data/trainval_labels.csv', 'data/trainval_bboxes.csv')

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)


In [ ]:
pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
for i in range(32):
  index = 3*i
  image,label = train_dataset[index]

In [ ]:
type(image)
image_name = './rob535-fall2021-final-project-data/trainval/'+file_name + '_image.jpg'

torch.Tensor